In [ ]:
!pip install -q optuna

    모델은 기본적으로 positive 클래스를 예측할 때 보수적 접근 방법으로 모델을 생성하기 때문에 너무 많은 위험을 감수하지 않는다.
    해결책은 positive 클래스에 더 많은 비중을 두도록 강제하는 hyperparameter를 활용하는 것이다.
    클래스 가중치를 부여할 수 있는 방법은 여러가지가 있지만, 그 중 하나인 'scale_pos_weight'를 활용.

    보통 많이 사용하는 값은 아래의 수식을 통해 계산한다.
    (number of negative samples) / (number of positive samples)

    보수적 접근 방법은 다음과 같은 특징 때문에 실제 적용이 너무 어려움.
    * True Positive(정탐)만 Maximize
    * False Positive(미탐) 최소화
    * 보통은 Recall이나 Precision을 더 선호
    * 미탐보다 오탐이 훨씬 더 많음

    현재 모델의 결과를 보면, 거의 모든 것이 Positive 클래스에 속할 것으로 예상되는 문제가 있다.

    이를 해결하기 위해 몇가지 제약 조건을 정의해야 한다.
    * 예를 들어, 재입원하지 않은 모든 데이터를 포함할 수 있도록 적어도 충분한 true negative(정음성)이 있어야 한다.

    * 또한 30일이 지나서 재입원한 환자도 False Positive에 포함되도록 비율을 조정해야 함.

In [ ]:
def optimize_lgb(trial):
    params = {
        'max_depth' : trial.suggest_int('max_depth', 2, 11),
        'scale_pos_weight' : trial.suggest_float('scale_pos_weight', def_scale_pos_weight/2, def_scale_pos_weight*2),
        'reg_lambda' : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'reg_alpha' : trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True)
    }
    if params['max_depth'] == 11:
        params['max_depth'] = -1

    # 모델 원형
    clf = lgb.LGBMClassifier(random_state=rand, n_jobs=-1, **params) # 이중 참조
    clf.fit(X_train, y_train)

    metrics_dict = evaluate_class_mdl(clf, X_train, X_test, y_train, y_test, plot=False)

    # optimize를 위한 최소조건을 준다는 의미
    if (metrics_dict['tn%'] < min_tn) or (metrics_dict['fp%'] > max_fp):
        return 0

    return metrics_dict['recall']

%%time
opt_study = optuna.create_study(direction='maximize')
opt_study.optimize(optimize_lgb, n_trials=100)

# best_params
opt_study.best_params